# **Gas-turbines**

In [ ]:
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import r2_score


In [ ]:
df = pd.read_csv('/content/gas_turbines.csv')
df.head()

# **Exploratory Data Analysis**

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.size

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
# Loop for getting Categorical data
for column in df.columns:
  if df[column].dtype == 'object':
    print(column)

In [ ]:
# Loop for getting Continuous data
for column in df.columns:
  if df[column].dtype != 'object':
    print(column)

# **Correlation Matrix**

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df.corr(),annot = True)
plt.show()

In [ ]:
Columns=['AT', 'AP', 'AH', 'AFDP', 'GTEP', 'TIT', 'TAT', 'TEY', 'CDP', 'CO','NOX']

In [ ]:
for feature in Columns:
  sns.displot(data = df, x = feature, height = 7,aspect = 2, color = '#158685')
  plt.show()

# **Box Plot**

In [ ]:
for col in Columns:
    plt.figure()   # plots figure for each iteration
    sns.boxplot(df[col])
    plt.show()

# **Dist Plot**

In [ ]:
for cols in Columns:
  plt.figure()
  sns.distplot(df[col])
  plt.show()

# **Violin Plot**

In [ ]:
for col in Columns:
  plt.figure()
  sns.violinplot(df[col])
  plt.show()

In [ ]:
df.drop(labels=['AT','AP','AH','NOX'],axis = 1, inplace = True)
df.head()

# **Defining Independent and Dependent Variable**

In [ ]:
X = df.drop('TEY',axis = 1)
Y = df['TEY']

In [ ]:
X.head()

In [ ]:
Y.head()

# **Scaling the data**

In [ ]:
# Standardization
a = StandardScaler()
a.fit(X)
X_standardized = a.transform(X)

In [ ]:
X_standardized.shape

In [ ]:
pd.DataFrame(X_standardized)

# **Splitting the data into Training and Testing dataset**

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

In [ ]:
X_train.shape,X_test.shape

# **Tuning the Hyperparameters :- Batch size and Epochs**

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import adam_v2

In [ ]:
# Create model
def create_model():
  model = Sequential()
  model.add(Dense(20, input_dim = 6, kernel_initializer='uniform',activation='relu')) 
  model.add(Dense(25, kernel_initializer='uniform',activation='relu'))
  model.add(Dense(1, kernel_initializer='uniform',activation='sigmoid'))

  adam = adam_v2.Adam(lr = 0.01)

  model.compile(loss = 'binary_crossentropy',
                optimizer = adam,
                metrics = ['accuracy'])
  
  return model

In [ ]:
# Create the model
model = KerasClassifier(build_fn = create_model, verbose = 0)


# Define the grid search parameter
batch_size = [10,20,30]
epochs = [30,40,50]

#Make a dictionary of the grid search parameters
param_grid = dict(batch_size = batch_size,
                  epochs = epochs)

# Build and fit the GridSearchCV
grid = GridSearchCV(estimator = model,
                    param_grid = param_grid,
                    cv = KFold(),
                    verbose = 10)

grid_result = grid.fit(X_standardized, Y)

In [ ]:
 # Summarize the results
print('Best : {}, using {}' .format(grid_result.best_score_,grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean,stdev,param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean,stdev, param))

# **Tuning of Hyperparameters:-Learning Rate and Drop out rate**

In [ ]:
from keras.layers import Dropout

# Defining the model
def create_model(learning_rate, dropout_rate):
  model1 = Sequential()
  model1.add(Dense(30, input_dim = 6, kernel_initializer='uniform',activation='relu'))
  model1.add(Dropout(dropout_rate))
  model1.add(Dense(25, input_dim = 6, kernel_initializer='uniform',activation='relu'))
  model1.add(Dropout(dropout_rate))
  model1.add(Dense(1,activation='sigmoid'))

  adam = adam_v2.Adam(lr = learning_rate)

  model1.compile(loss = 'binary_crossentropy',
                optimizer = adam,
                metrics = ['accuracy'])
  return model1

In [ ]:
# Create the model
model1 = KerasClassifier(build_fn = create_model, 
                        verbose = 0,
                        batch_size = 10,
                        epochs = 30)


# Define the grid search parameter
learning_rate = [0.001,0.01,0.1]
dropout_rate= [0.1,0.2,0.3]

#Make a dictionary of the grid search parameters
param_grid = dict(learning_rate = learning_rate,
                  dropout_rate = dropout_rate)

# Build and fit the GridSearchCV
grid = GridSearchCV(estimator = model1,
                    param_grid = param_grid,
                    cv = KFold(),
                    verbose = 10)

grid_result = grid.fit(X_standardized, Y)

In [ ]:
# Summarize the results
print('Best : {}, using {}' .format(grid_result.best_score_,grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean,stdev,param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean,stdev, param))

# **Tuning of Hyperparameters:- Activation Function and Kernel Initializer**

In [ ]:
# Defining the model

def create_model(activation_function,init):
    model2 = Sequential()
    model2.add(Dense(30,input_dim = 6,kernel_initializer = init,activation = activation_function))
    model2.add(Dropout(0.1))
    model2.add(Dense(25,input_dim = 6,kernel_initializer = init,activation = activation_function))
    model2.add(Dropout(0.1))
    model2.add(Dense(1,activation = 'sigmoid'))
    
    adam = adam_v2.Adam(lr = 0.001)
    model2.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model2

# Create the model

model2 = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 10,epochs = 30)

# Define the grid search parameters
activation_function = ['softmax','relu','tanh','linear']
init = ['uniform','normal','zero']

# Make a dictionary of the grid search parameters
param_grids = dict(activation_function = activation_function,init = init)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model2,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,Y)

In [ ]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

#**Tuning of Hyperparameter :-Number of Neurons in activation layer**

In [ ]:
# Defining the model

def create_model(neuron1,neuron2):
    model3 = Sequential()
    model3.add(Dense(neuron1,input_dim = 6,kernel_initializer = 'uniform',activation = 'linear'))
    model3.add(Dropout(0.1))
    model3.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = 'uniform',activation = 'linear'))
    model3.add(Dropout(0.1))
    model3.add(Dense(1,activation = 'sigmoid'))
    
    adam = adam_v2.Adam(lr = 0.001)
    model3.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model3

# Create the model

model3 = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 10,epochs = 30)

# Define the grid search parameters

neuron1 = [4,8,16]
neuron2 = [2,4,8]

# Make a dictionary of the grid search parameters

param_grids = dict(neuron1 = neuron1,neuron2 = neuron2)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model3,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,Y)

In [ ]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

# **Training model with optimum values of Hyperparameters**

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Defining the model

def create_model():
    model4 = Sequential()
    model4.add(Dense(4,input_dim = 6,kernel_initializer = 'uniform',activation = 'linear'))
    model4.add(Dropout(0.1))
    model4.add(Dense(2,input_dim = 4,kernel_initializer = 'uniform',activation = 'linear'))
    model4.add(Dropout(0.1))
    model4.add(Dense(1,activation = 'sigmoid'))
    
    adam = adam_v2.Adam(lr = 0.001) #sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
    model4.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model4

# Create the model

model4 = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 10,epochs = 30)

# Fitting the model

model4.fit(X_standardized,Y)

# Predicting using trained model

Y_predict = model4.predict(X_standardized)

# Printing the metrics
print(r2_score(Y,Y_predict))

# **Building Neural Networks Model using Optimal Values**

In [ ]:
 # create ANN model
model = Sequential()
# Defining the first layer of the model
model.add(Dense(units=4, input_dim=X_train.shape[1], kernel_initializer='uniform', activation='linear'))
model.add(Dropout(0.1))        
# Defining the Second layer of the model
model.add(Dense(units=2, kernel_initializer='uniform', activation='linear'))
model.add(Dropout(0.1))  
# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy', metrics=['accuracy'])

# Training the model with best parameters
history = model.fit(X_train, Y_train, validation_split=0.33, batch_size = 10, epochs = 30)

In [ ]:
# Evaluate the model
scores = model.evaluate(X,Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
model.metrics_names

In [ ]:
scores

In [ ]:
#visualize training history

#list all data in history
history.history.keys()

In [ ]:
#Summarize history for accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
#Summarize history for loss
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()